In [11]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load data

In [2]:
import pandas as pd
a = pd.read_feather("ratings5.feather")
a.head()

,index,user,anime_id,score,status,created_at
0,0,DeleteAccountPlz,12149,8,2,1337262717
1,1,DeleteAccountPlz,14941,9,2,1365094767
2,2,DeleteAccountPlz,150,10,2,1401970697
3,3,DeleteAccountPlz,1482,10,2,1401970764
4,4,DeleteAccountPlz,1535,10,2,1365095869


In [3]:
a[a['user'] == "DeleteAccountPlz"]

,index,user,anime_id,score,status,created_at
0,0,DeleteAccountPlz,12149,8,2,1337262717
1,1,DeleteAccountPlz,14941,9,2,1365094767
2,2,DeleteAccountPlz,150,10,2,1401970697
3,3,DeleteAccountPlz,1482,10,2,1401970764
4,4,DeleteAccountPlz,1535,10,2,1365095869
5,5,DeleteAccountPlz,122,10,2,1327245438
6,6,DeleteAccountPlz,5114,10,2,1327245313
7,7,DeleteAccountPlz,1281,9,2,1401970752
8,8,DeleteAccountPlz,1516,9,2,1401970723
9,9,DeleteAccountPlz,2034,8,2,1401970936


In [13]:
a.drop(columns=["index"], inplace=True)
a.dtypes

user          object
anime_id       int32
score           int8
status          int8
created_at     int32
dtype: object

### Count "popular" items

In [14]:
a = a[a['status'] < 5]
zero_users = a[a['status']==0].loc[:,'user'].unique()
a = a[~a['user'].isin(zero_users)]   

In [15]:
b = a['anime_id'].value_counts()
b[b>300].count()

10654

In [16]:
a.rename(columns={'user':'username'},inplace=True)

### Remove rare items

In [17]:
a = a[a['anime_id'].isin(b[b>300].index)]
a.shape

(200059494, 5)

### Count and remove rare users

In [18]:
b = a['username'].value_counts()
b[b>5].count()

1086472

In [19]:
a = a[a['username'].isin(b[b>5].index)]
a.shape

(199874036, 5)

### Map usernames to numbers

In [20]:
mapper = dict(zip(a['username'].unique(),[*range(a['username'].nunique())]))
a['username'] = a['username'].map(mapper)
a['username'].unique()

array([      0,       1,       2, ..., 1086469, 1086470, 1086471],
      dtype=int64)

### Remove extra columns

In [21]:
a = a.iloc[:,:].set_index('username').astype("int32")

In [22]:
a.head()

,anime_id,score,status,created_at
username,,,,
0,12149,8,2,1337262717
0,14941,9,2,1365094767
0,150,10,2,1401970697
0,1482,10,2,1401970764
0,1535,10,2,1365095869


### Remap items and store original item -> new item mapping in csv

In [23]:
# b = pd.DataFrame(a.loc[:,"anime_id"])

uniques = a['anime_id'].unique()
mapper = dict(zip(uniques,range(len(uniques))))
# d = a['anime_id'].map(mapper)
# b['train_id'] = d

c = pd.DataFrame(zip(uniques,[*range(len(uniques))]),columns=['item_id','train_id'])
c.to_csv("item_map.csv",index=False)
a['anime_id'] = a['anime_id'].map(mapper)
# a.head()
c.shape

(10654, 2)

In [24]:
e = a.groupby("username",sort=False)["score"]
f = e.transform(pd.Series.median)

In [25]:
del b,c,mapper
a = a[(f>0)& (f<10)]

In [26]:
e = a.groupby("username",sort=False)["score"]
g = e.transform(pd.Series.quantile,q=0.25,interpolation="nearest")

In [27]:
(g>0).sum()/len(g)

0.83834687187064

In [28]:
a = a[(g>0)]

In [29]:
import numpy as np


In [30]:
e = a.groupby("username",sort=False)["score"]
f = e.transform(pd.Series.median)
a['train_label'] = 0
a['train_label'] += np.bitwise_and((a['score']-f)>0, a['status']<=3)

In [31]:
a.head(n=15)

,anime_id,score,status,created_at,train_label
username,,,,,
2,323,7,1,0,0
2,324,7,1,0,0
2,325,0,1,0,0
2,326,0,1,0,0
2,327,7,1,0,0
2,328,8,1,0,1
2,329,9,1,0,1
2,330,8,2,0,1
2,77,8,2,0,1


In [32]:
a.drop(columns=['index'],inplace=True)

KeyError: "['index'] not found in axis"

In [ ]:
a.reset_index().to_feather("ratings5_processed.feather")

In [34]:
groups = a.groupby("username",sort=False)
diff = lambda x: x.max()-x.min()
diff2 = lambda x: (x.shift(1,fill_value=x.iloc[0])-x).mean()


In [23]:
groups.count().min() # 6 is good

anime_id       6
score          6
status         6
created_at     6
train_label    6
dtype: int64

In [36]:
column_funcs = {'created_at': [lambda x: diff(x),lambda x: diff2(x),pd.Series.count,pd.Series.std],"score":[pd.Series.mean,pd.Series.std]}
time_stats = groups.agg(column_funcs)

In [73]:
a[a.index==108].tail()

,anime_id,score,status,created_at,train_label
username,,,,,
108,411,8,2,0,0
108,1044,8,2,0,0
108,340,0,3,0,0
108,1718,0,3,0,0
108,1524,0,3,0,0


In [56]:
time_stats.head()

created_at                                        score          
         <lambda_0>     <lambda_1> count           std      mean       std
username                                                                  
2                 0       0.000000   127  0.000000e+00  6.929134  2.086164
5          50784357   -4361.058394   137  1.708781e+07  7.459854  1.514705
6         192273697  563525.479167    96  4.836136e+07  7.500000  2.082509
8          85817504 -157390.372093   172  2.406463e+07  7.924419  1.816135
9           9749346  255861.240000    25  3.303754e+06  7.040000  2.791057

In [57]:
time_stats.columns = time_stats.columns.get_level_values(1)

In [68]:
time_stats.rename(columns={'<lambda_0>':'range','<lambda_1>':'time_diffs'},inplace=True)
time_stats.head()

,range,time_diffs,count,std,mean,std
username,,,,,,
2,0,0.000000,127,0.000000e+00,6.929134,2.086164
5,50784357,-4361.058394,137,1.708781e+07,7.459854,1.514705
6,192273697,563525.479167,96,4.836136e+07,7.500000,2.082509
8,85817504,-157390.372093,172,2.406463e+07,7.924419,1.816135
9,9749346,255861.240000,25,3.303754e+06,7.040000,2.791057


In [74]:
#normalize every column of time_stats
time_stats_norm = (time_stats-time_stats.mean())/time_stats.std()
time_stats_norm.head()

,range,time_diffs,count,std,mean,std
username,,,,,,
2,-0.545736,-0.099782,-0.287913,-0.554695,-0.208274,-0.351096
5,-0.399935,-0.101894,-0.239512,-0.337434,0.389937,-1.094735
6,0.006279,0.173112,-0.437954,0.060192,0.435188,-0.355853
8,-0.299355,-0.176001,-0.070111,-0.248727,0.913579,-0.702484
9,-0.517745,0.024122,-0.781596,-0.512690,-0.083309,0.566181


In [76]:
item_popularities = a['anime_id'].value_counts()
pop_mean=lambda x: item_popularities[x].mean()
pop_std = lambda x: item_popularities[x].std()
popularity_stats = groups.agg({"anime_id":[pop_mean,pop_std]})

In [77]:
popularity_stats.head()

anime_id               
             <lambda_0>     <lambda_1>
username                              
2         107712.031496   76938.692872
5         145190.635036  105792.059632
6         151433.958333  126928.904286
8          63286.732558   67068.216998
9         102096.800000  131031.882809

In [78]:
popularity_stats.columns = popularity_stats.columns.get_level_values(1)
popularity_stats.rename(columns={'<lambda_0>':'pop_mean','<lambda_1>':'pop_std'},inplace=True)
popularity_stats.head()

,pop_mean,pop_std
username,,
2,107712.031496,76938.692872
5,145190.635036,105792.059632
6,151433.958333,126928.904286
8,63286.732558,67068.216998
9,102096.800000,131031.882809


In [90]:
all_stats = pd.concat([time_stats,popularity_stats],axis=1)
all_stats.head()

,range,time_diffs,count,std,mean,std,pop_mean,pop_std
username,,,,,,,,
2,0,0.000000,127,0.000000e+00,6.929134,2.086164,107712.031496,76938.692872
5,50784357,-4361.058394,137,1.708781e+07,7.459854,1.514705,145190.635036,105792.059632
6,192273697,563525.479167,96,4.836136e+07,7.500000,2.082509,151433.958333,126928.904286
8,85817504,-157390.372093,172,2.406463e+07,7.924419,1.816135,63286.732558,67068.216998
9,9749346,255861.240000,25,3.303754e+06,7.040000,2.791057,102096.800000,131031.882809


In [91]:
j = 0
cols = all_stats.columns.values.copy()
for i in range(len(cols)):
    if cols[i] == "std":
        if j:
            cols[i] = "score_std"
        else:
            cols[i] = "time_std"
        j+= 1
all_stats.columns = cols
print(all_stats.columns)
        

Index(['range', 'time_diffs', 'count', 'time_std', 'mean', 'score_std',
       'pop_mean', 'pop_std'],
      dtype='object')


In [92]:
all_stats.reset_index().to_feather("user_stats.feather")

In [ ]:
e = a.groupby("username",sort=False)["score"].transform(pd.Series.mean)
a = a[(e>0)& (e<10)] #(a['score'] >= e) & 
a.shape

MemoryError: Unable to allocate 973. MiB for an array with shape (255189804,) and data type int32

### Split data

In [ ]:
import pandas as pd
import numpy as np
a_small = pd.read_feather("ratings5_processed.feather")

In [ ]:
a_small

,username,index,anime_id,score,status,created_at,train_label
0,2,327,323,7,1,0,0
1,2,328,324,7,1,0,0
2,2,329,325,0,1,0,0
3,2,330,326,0,1,0,0
4,2,331,327,7,1,0,0


In [ ]:
a = pd.read_feather("ratings5.feather")

In [ ]:
a[np.bitwise_and(a['status']==2,a['score']==0)]

,index,user,anime_id,score,status,created_at
68,68,Emiya9901,6682,0,2,1520456043
69,69,Emiya9901,11759,0,2,1520453747
70,70,Emiya9901,31580,0,2,1520455186
71,71,Emiya9901,33253,0,2,1520457963
72,72,Emiya9901,22199,0,2,1520453391
...,...,...,...,...,...,...
256206749,212501301,YaraAlasaad,543,0,2,1438210121
256206750,212501302,YaraAlasaad,2608,0,2,1438210130
256206751,212501303,YaraAlasaad,861,0,2,1438210147
256206752,212501304,YaraAlasaad,793,0,2,1438210156


In [ ]:
a_small[a_small['status']==0]

,username,anime_id,score,status,created_at,train_label
1892149,13081,3602,0,0,0,0
2235468,15468,5907,9,0,0,1
2235469,15468,3896,9,0,0,1
2235470,15468,7876,8,0,0,0
2235471,15468,9568,0,0,0,0
...,...,...,...,...,...,...
150951343,1001490,4600,7,0,0,1
151315722,1004412,827,7,0,1644366591,0
151315723,1004412,931,9,0,1644400606,1
151315724,1004412,932,9,0,1644400630,1


In [ ]:
validation = np.random.choice(a_small.index.unique(),size=90000,replace=False)
validation_df = a_small[a_small.index.isin(validation[len(validation)//2:])]
test_df = a_small[a_small.index.isin(validation[:len(validation)//2])]
train = a_small[~a_small.index.isin(validation)]
train.shape,validation_df.shape,test_df.shape

((45358603, 3), (2764379, 3), (2786160, 3))

### Store and load data

In [ ]:
train.reset_index().to_feather("train_processed.feather")
test_df.reset_index().to_feather("test_processed.feather")
validation_df.reset_index().to_feather("valid_processed.feather")

In [ ]:
p = pd.read_feather("data/train_processed.feather")
p['score'].min(),p['score'].max(),p['anime_id'].max()

(3, 10, 10174)

In [ ]:
p['username'].nunique()

489803

In [ ]:
import torch
a = torch.tensor([[1,0,3],[0,5,6]])
b = torch.arange(2,5)
a/b

tensor([[0.5000, 0.0000, 0.7500],
        [0.0000, 1.6667, 1.5000]])

In [ ]:
c = torch.tensor([[1,8,3],[9,5,6]])
c[b[:,0],b[:,1]] = 100
c

tensor([[100,   8, 100],
        [  9, 100, 100]])

## Reformatting item map to contain anime titles

In [ ]:
import pandas as pd
import sqlite3
item_map = pd.read_csv("item_map.csv").set_index("item_id")
table = pd.read_sql("select id,anime_title from anime_info",con=sqlite3.connect("../DiscordBot/animeinfo.sqlite3")).set_index("id")
# item_map.join(table).to_csv("item_map.csv")

In [ ]:
%timeit -n1 -r 2 item_map.reindex([*range(5000)]).dropna()['train_id'].values
%timeit -n1 -r 2 item_map.loc[item_map.index.isin([*range(5000)]),"train_id"].values


1.17 ms ± 231 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)
468 µs ± 87.4 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [ ]:
import torch
a = torch.rand(100)
a[[1,30]] = -(2**31)
out_sorted = torch.argsort(a,descending=True)
out_sorted

tensor([ 2, 25, 87, 41, 77, 61, 23, 86, 38, 58, 17, 21, 19,  4, 50, 69, 44, 88,
        28, 60, 32, 71, 52, 26, 91, 96, 34, 63, 11, 56, 66, 31, 81, 68, 65,  0,
        24, 83,  7, 35, 13, 53, 85, 54, 98, 55, 14, 67, 47, 92, 36, 15,  9, 43,
        99, 27, 89,  6, 20, 46, 95, 84, 79, 12, 94, 78, 72, 49, 76, 57, 80, 59,
        82, 74, 22, 97, 16, 75,  8, 42, 10, 40, 45, 29, 18, 62, 39, 70, 93, 48,
        73, 37, 33, 90, 64, 51,  3,  5,  1, 30])

In [ ]:
from scipy import stats
import numpy as np

In [ ]:
a = np.random.normal(0,1,10)
a = np.arange(10)
b = a[::-1]
stats.spearmanr(a,b)

SpearmanrResult(correlation=-0.9999999999999999, pvalue=6.646897422032013e-64)